In [2]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras_preprocessing import sequence
from keras.utils import to_categorical
from keras.models import load_model

import csv
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import nltk
nltk.download('stopwords')  
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

from tqdm.auto import tqdm

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
df = pd.read_csv('/content/spam.csv',delimiter=',',encoding='latin-1')
df.drop(['Unnamed: 2','Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True) 
df.sample(5)

,v1,v2
1577,ham,Remember all those whom i hurt during days of ...
4871,ham,Hi dis is yijue i would be happy to work wif Ì...
1476,ham,I'm watching lotr w my sis dis aft. So u wan 2...
3572,spam,You won't believe it but it's true. It's Incre...
1590,ham,Somewhr someone is surely made 4 u. And God ha...


In [22]:
df.rename(columns = {'v1' : 'label', 'v2' : 'text'}, inplace = True)
df.sample(2)

,label,text
4230,ham,U really pig leh sleep so much. My dad wake me...
1960,spam,Guess what! Somebody you know secretly fancies...


In [5]:
print("number of rows : ", df.shape[0])
print("number of columns : ", df.shape[1])

number of rows :  5572
number of columns :  2


In [7]:
#Label Encoding Required Column
X = df.text
y = df.label

le = LabelEncoder()
y = le.fit_transform(y)
y = y.reshape(-1, 1)
# Test and train data split 
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    test_size = 0.2,
                                                    random_state = 42,
                                                    shuffle = True)

In [8]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words = max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences, maxlen=max_len)

In [9]:
#LSTM model
inputs = Input(name='InputLayer',shape=[max_len])
layer = Embedding(max_words, 50, input_length = max_len)(inputs)
layer = LSTM(64)(layer)
layer = Dense(256, name='FullyConnectedLayer1')(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1,name='OutputLayer')(layer)
layer = Activation('sigmoid')(layer)

In [11]:
model = Model(inputs = inputs, outputs = layer)
model.summary()
model.compile(loss='binary_crossentropy', 
              optimizer = 'adam',
              metrics = ['accuracy'])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 InputLayer (InputLayer)     [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FullyConnectedLayer1 (Dense  (None, 256)              16640     
 )                                                               
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                           

In [12]:
model.fit(sequences_matrix, 
          y_train,
          batch_size = 64,
          epochs = 20,
          validation_split = 0.2)

Epoch 1/20
56/56 [==============================] - 11s 130ms/step - loss: 0.3160 - accuracy: 0.8926 - val_loss: 0.0948 - val_accuracy: 0.9809
Epoch 2/20
56/56 [==============================] - 7s 119ms/step - loss: 0.0511 - accuracy: 0.9857 - val_loss: 0.0462 - val_accuracy: 0.9854
Epoch 3/20
56/56 [==============================] - 7s 119ms/step - loss: 0.0265 - accuracy: 0.9921 - val_loss: 0.0508 - val_accuracy: 0.9843
Epoch 4/20
56/56 [==============================] - 7s 134ms/step - loss: 0.0140 - accuracy: 0.9958 - val_loss: 0.0491 - val_accuracy: 0.9821
Epoch 5/20
56/56 [==============================] - 8s 141ms/step - loss: 0.0072 - accuracy: 0.9986 - val_loss: 0.0568 - val_accuracy: 0.9832
Epoch 6/20
56/56 [==============================] - 8s 143ms/step - loss: 0.0053 - accuracy: 0.9992 - val_loss: 0.0709 - val_accuracy: 0.9832
Epoch 7/20
56/56 [==============================] - 7s 119ms/step - loss: 0.0034 - accuracy: 0.9992 - val_loss: 0.0721 - val_accuracy: 0.9798
Epoch

In [13]:
model.save("sms_spam")

In [14]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,
                                               maxlen = max_len)

accuracy = model.evaluate(test_sequences_matrix, 
                           y_test)

print('Accuracy: {:0.3f}'.format(accuracy[1]))

35/35 [==============================] - 1s 22ms/step - loss: 0.1829 - accuracy: 0.9776
Accuracy: 0.978


In [15]:
y_pred = model.predict(test_sequences_matrix)

35/35 [==============================] - 2s 34ms/step


In [19]:
np.round(y_pred[:5])

array([[1.],
       [0.],
       [1.],
       [0.],
       [1.]], dtype=float32)

In [20]:
y_test[:5]

array([[0],
       [0],
       [1],
       [0],
       [1]])